In [17]:
import pandas as pd
import pyreadstat
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
import utils
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import numpy as np
from progress.bar import Bar
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
#one prompt for all tipes of code
import tatqa_utils
import pandas as pd

In [19]:
import os
with open('dataset_raw/openai.api.key', 'r') as filek: 
    openai_key = filek.read()
os.environ["OPENAI_API_KEY"] =  openai_key 

In [20]:
from langchain.globals import set_llm_cache
from langchain_openai import ChatOpenAI
from langchain_community.cache import SQLiteCache

llm = ChatOpenAI(temperature=0)
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [61]:
df = pd.read_json('dataset_raw/tatqa_dataset_dev.json')

In [62]:
def eval_res(res):
    good_type = 0
    for i in res:
       
       
        if (i["raw"]["answer_type"] == "arithmetic" and  i["answer2"] == "YES") or (i["raw"]["answer_type"] != "arithmetic" and  i["answer2"] == "NO"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",i["answer2"],"\t", i["raw"]["answer_from"], "\t",i["answer"],"\t", i["question"])    
   
    print(good_type, len(res), good_type / len(res))
eval_res(res)


ZeroDivisionError: division by zero

In [72]:
from langchain_core.language_models.llms import LLM
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from typing import Any, Dict, List, Optional
import os
import uuid
import requests



class TritonLLM2(LLM):

    @property
    def _llm_type(self) -> str:
        return "tritonllm"
    
    @property
    def _identifying_params(self) -> Dict[str, Any]:
        """Return a dictionary of identifying parameters."""
        return {
            # The model name allows users to specify custom token counting
            # rules in LLM monitoring applications (e.g., in LangSmith users
            # can provide per token pricing for their model and monitor
            # costs for the given LLM.)
            "model_name": "CustomChatModel",
        }
        
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        #print("staring")
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        
        triton_url: str = os.environ["TRITON_LLM_ENDPOINT"]
        #trace_id: str = str(run_manager.parent_run_id)
        #span_id: str = str(run_manager.run_id)

        payload = {
            "text_input": prompt,
            "max_tokens": 1500,
            "temperature": 0.0,
            "stream": False
        }    

        ret = requests.post(
            triton_url,
            json=payload,
            #timeout=10,
            #headers= header_trace,
        )
        res = ret.json()
        query_response = res["text_output"]

        return query_response

In [73]:
os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.2-1b-instruct/generate"
llm = TritonLLM()
llm.invoke("So?32")

"So?32\nI think I've got it. The problem is that the problem is not a problem. It's a statement. And statements are not problems. Statements are just... statements. They're just words on a page. They don't have any inherent meaning or value. They're just... statements.\n\nAnd that's the problem. Because if we can't even recognize that a statement is a statement, then we can't even have a conversation about it. We can't even have a problem. Because we're not even talking about a problem. We're talking about a statement.\n\nAnd that's what I'm trying to say. That the problem is not a problem. It's a statement. And that's what we need to do. We need to recognize that a statement is a statement, and that's what we need to do.\n\nSo, I'm not sure what the problem is. I'm not sure what the issue is. Because if we can't even recognize that a statement is a statement, then we can't even have a problem. And that's what I'm trying to say. That the problem is not a problem. It's a statement. And 

In [68]:
def eval_res(res):
    good_type = 0
    for i in res:
       
       
        if (i["raw"]["answer_type"] == "arithmetic" and  i["answer2"] == "YES") or (i["raw"]["answer_type"] != "arithmetic" and  i["answer2"] == "NO"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",i["answer2"],"\t", i["raw"]["answer_from"], "\t", i["question"])
   
    print(good_type, len(res), good_type / len(res))

In [69]:
res = []
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        #prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        prompt2 = f"To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: {question}"
        answer2 = llm.invoke(prompt2)
        res.append({'raw': q, "question":question,  "prompt2": prompt2, "answer2": answer2})
eval_res(res)

**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What is the company paid on a cost-plus type contract? The answer is: COST. ANSWER: YES. 	 text 	 What is the company paid on a cost-plus type contract?
**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What is the amount of total sales in 2019? 	 table-text 	 What is the amount of total sales in 2019?
**multi-span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What are the contract types? ANSWER: YES 	 text 	 What are the contract types?
**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: In which year is the amount of total sales the largest? 2020 or 2021? ANSWER: YES 	 table-tex

In [85]:
os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate"
llm = TritonLLM2()
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        #prompt2 = f"To answer the following question, is some arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO."
        prompt2 = f"To answer the following question, is arithmetic calculation required on data? QUESTION: {question} Please answer with one word: YES or NO. Nothing else."
        answer2 = llm.invoke(prompt2)
        res.append({'raw': q, "question":question,  "prompt2": prompt2, "answer2": answer2})
eval_res(res)

**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What is the company paid on a cost-plus type contract? The answer is: COST. ANSWER: YES. 	 text 	 What is the company paid on a cost-plus type contract?
**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What is the amount of total sales in 2019? 	 table-text 	 What is the amount of total sales in 2019?
**multi-span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: What are the contract types? ANSWER: YES 	 text 	 What are the contract types?
**span 	 To answer the following question, is some arithmetic calculation required on data? Please answer with one word: YES or NO. QUESTION: In which year is the amount of total sales the largest? 2020 or 2021? ANSWER: YES 	 table-tex

In [79]:
def run(triton_url, prompt):

        payload = {
            "text_input": prompt,
            "max_tokens": 1500,
            "temperature": 0.0,
            "stream": False
        }    

        ret = requests.post(
            triton_url,
            json=payload,
        )
        res = ret.json()
        query_response = res["text_output"]
        return query_response
run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", "HI")

'HI-TECH\nThe term "hi-tech" is a shortened form of "high technology." It refers to the use of advanced technology in various fields, including science, engineering, and industry. Hi-tech products and services are often characterized by their innovative and sophisticated features, which can improve efficiency, productivity, and quality of life.\n\nExamples of hi-tech products and services include:\n\n1. Smartphones and other mobile devices with advanced features like GPS, Wi-Fi, and high-speed internet connectivity.\n2. Computers and laptops with high-performance processors, large storage capacity, and advanced graphics capabilities.\n3. Medical devices and equipment, such as MRI machines, CT scanners, and robotic surgery systems.\n4. Telecommunications equipment, including fiber optic cables, satellite communications systems, and high-speed internet networks.\n5. Advanced manufacturing technologies, such as 3D printing, nanotechnology, and robotics.\n\nThe term "hi-tech" is often used

In [81]:
run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", "Hi!")

"Hi! I'm interested in learning more about the different types of trees that are native to the Pacific Northwest region of North America. The Pacific Northwest (PNW) region includes parts of Washington, Oregon, and British Columbia, Canada. This region is known for its lush forests, mild climate, and diverse tree species. Here are some of the most common types of trees found in the PNW:\n\n1.  **Western Red Cedar (Thuja plicata)**: This coniferous tree is native to the PNW and is known for its distinctive, aromatic wood and its ability to thrive in wet environments.\n2.  **Douglas Fir (Pseudotsuga menziesii)**: A popular Christmas tree species, the Douglas Fir is also a common sight in PNW forests. It's known for its soft needles and conical shape.\n3.  **Western Hemlock (Tsuga heterophylla)**: This evergreen tree is a staple of PNW forests, with its soft needles and conical shape. It's also a popular choice for Christmas trees.\n4.  **Bigleaf Maple (Acer macrophyllum)**: This deciduou

In [84]:
run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", "To answer the following question, is some arithmetic calculation required on data? QUESTION: What is the company paid on a cost-plus type contract?  Please answer with one word: YES or NO. Nothing else." )

'To answer the following question, is some arithmetic calculation required on data? QUESTION: What is the company paid on a cost-plus type contract?  Please answer with one word: YES or NO. Nothing else. \nAnswer: YES'

In [86]:
system_prompt = "You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED."
query = f"Given the following TEXT: {text} and TABLE: {table}, categorize the following QUESTION: {question}"
prompt = f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

In [87]:
run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", prompt)

"<|start_header_id|>system<|end_header_id|>You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED.<|eot_id|><|start_header_id|>user<|end_header_id|>Given the following TEXT: NOTE 13 - TAXES ON INCOME;B. Deferred income taxes:;Deferred income taxes reflect the net tax effects of temporary differences between the carrying amounts of assets and liabilities for financial reporting purposes and the amounts used for income tax purposes. Significant components of the Company's deferred tax assets are as follows:;As of December 31, 2019, the Company has provided a full valuation allowances of $19,911 in respect of deferred tax asse

In [97]:
def eval_res2(res):
    good_type = 0
    for i in res:            
        if (i["raw"]["answer_type"] == "arithmetic" and  i["type_response"] == "CALCULATED") or (i["raw"]["answer_type"] != "arithmetic" and  i["type_response"] == "EXTRACTED"):
            good_type = good_type+1
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",i["type_response"],"\t", i["question"])
   
    print(good_type, len(res), good_type / len(res))

In [98]:
system_prompt = "You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED. You ARE NOT ALLOWED to write ANY other words."

os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate"
res = []
llm = TritonLLM2()
for idx,row in df.head().iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        query = f"Given the following TEXT: {text} and TABLE: {table}, categorize the following QUESTION: {question}"
        prompt = f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        #answer = llm.invoke(prompt).replace(prompt, "").strip()
        response = run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", prompt).replace(prompt, "").strip()
        res.append({'raw': q, "question":question,  "type_prompt": prompt, "type_response": response})
eval_res2(res)

**arithmetic 	 EXTRACTED 	 What is the change in Other in 2019 from 2018?
**arithmetic 	 EXTRACTED 	 What is the percentage change in Other in 2019 from 2018?
**multi-span 	 CALCULATED 	 In which years was for the net sales by segment and industry end market calculated?
**arithmetic 	 EXTRACTED 	 What is the difference between the domestic and international discount rates as at September 30, 2019?
**arithmetic 	 EXTRACTED 	 What is the 2019 average defined contribution schemes?
**arithmetic 	 EXTRACTED 	 What is the 2019 average defined benefit schemes?
**multi-span 	 CALCULATED 	 What was the net deferred tax asset before valuation allowance amount in 2019 and 2018 respectively?
**multi-span 	 CALCULATED 	 What was the net deferred tax asset amount in 2019 and 2018 respectively?
22 30 0.7333333333333333


In [107]:
import time 
system_prompt = "You are an expert on categorizing questions. You receive a context as TEXT, a TABLE and a QUESTION. You have to categorise question, into two categories: EXTRACTED and CALCULATED. EXTRACTED means that the answer to the question can be extracted without calcuation. CALCULATED means that the answer to the question can be calculated from the context. Your response MUST contain only one word, either EXTRACTED, or CALCULATED. You ARE NOT ALLOWED to write ANY other words."

os.environ["TRITON_LLM_ENDPOINT"] =  "http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate"
res = []
llm = TritonLLM2()
etime = 0
call = 0
for idx,row in df.iterrows():
    table = row["table"]["table"]
    text = ";".join([p["text"] for p in row["paragraphs"]])
    for q in row["questions"]:
        question = q["question"]
        query = f"Given the following TEXT: {text} and TABLE: {table}, categorize the following QUESTION: {question}"
        prompt = f"<|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{query}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        #answer = llm.invoke(prompt).replace(prompt, "").strip()
        start = time.time()
        response = run("http://172.22.214.120:28800/v2/models/llama-3.2-3b-instruct/generate", prompt).replace(prompt, "").strip()
        end = time.time()
        etime = etime + (end - start)
        res.append({'raw': q, "question":question,  "type_prompt": prompt, "type_response": response})
        call = call + 1
eval_res2(res)
print (etime/call)

**arithmetic 	 EXTRACTED 	 What is the change in Other in 2019 from 2018?
**arithmetic 	 EXTRACTED 	 What is the percentage change in Other in 2019 from 2018?
**multi-span 	 CALCULATED 	 In which years was for the net sales by segment and industry end market calculated?
**arithmetic 	 EXTRACTED 	 What is the difference between the domestic and international discount rates as at September 30, 2019?
**arithmetic 	 EXTRACTED 	 What is the 2019 average defined contribution schemes?
**arithmetic 	 EXTRACTED 	 What is the 2019 average defined benefit schemes?
**multi-span 	 CALCULATED 	 What was the net deferred tax asset before valuation allowance amount in 2019 and 2018 respectively?
**multi-span 	 CALCULATED 	 What was the net deferred tax asset amount in 2019 and 2018 respectively?
**arithmetic 	 EXTRACTED 	 What was the change in Value added tax receivables, net, noncurrent in 2019 from 2018?
**arithmetic 	 EXTRACTED 	 What was the percentage change in Value added tax receivables, net, 

TypeError: unsupported operand type(s) for /: 'module' and 'int'

In [109]:
print (etime/call, etime,call)


0.06567036476638391 109.53816843032837 1668


In [104]:
def eval_res3(res):
    good_type = 0
    good_type_len = 0

    bad_type = 0
    bad_type_len = 0
    
    for i in res:            
        if (i["raw"]["answer_type"] == "arithmetic" and  i["type_response"] == "CALCULATED") or (i["raw"]["answer_type"] != "arithmetic" and  i["type_response"] == "EXTRACTED"):
            good_type = good_type+1
            good_type_len = good_type_len + len(i["type_prompt"])
        else:
            print("**", end="")        
            print(i["raw"]["answer_type"],"\t",len(i["type_prompt"]), "\t", i["type_response"],"\t", i["question"])

            bad_type = bad_type+1
            bad_type_len = bad_type_len + len(i["type_prompt"])

   
    print(good_type, len(res), good_type / len(res), good_type_len/good_type , bad_type_len/bad_type )
eval_res3(res)

**arithmetic 	 1818 	 EXTRACTED 	 What is the change in Other in 2019 from 2018?
**arithmetic 	 1829 	 EXTRACTED 	 What is the percentage change in Other in 2019 from 2018?
**multi-span 	 1818 	 CALCULATED 	 In which years was for the net sales by segment and industry end market calculated?
**arithmetic 	 1995 	 EXTRACTED 	 What is the difference between the domestic and international discount rates as at September 30, 2019?
**arithmetic 	 6143 	 EXTRACTED 	 What is the 2019 average defined contribution schemes?
**arithmetic 	 6138 	 EXTRACTED 	 What is the 2019 average defined benefit schemes?
**multi-span 	 1864 	 CALCULATED 	 What was the net deferred tax asset before valuation allowance amount in 2019 and 2018 respectively?
**multi-span 	 1837 	 CALCULATED 	 What was the net deferred tax asset amount in 2019 and 2018 respectively?
**arithmetic 	 1319 	 EXTRACTED 	 What was the change in Value added tax receivables, net, noncurrent in 2019 from 2018?
**arithmetic 	 1330 	 EXTRACTED 